In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm

## investigating volatility linkages between oil, gas, and regionalised coal futures markets ##

## load data from Github into new frame and remove rows with absent data ##

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/df-combined.csv'
df = pd.read_csv(url)
df["DATE"] = pd.to_datetime(df["DATE"],dayfirst=True)
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col],errors='coerce')

df = df.dropna()
print(df)

           DATE    NCF    RCF    WTI    HH  NBP_UK  GBP-USD
1    2009-01-02  74.35  81.00  46.17  5.41   54.74   1.4453
4    2009-01-05  77.65  84.40  48.61  5.83   56.66   1.4708
5    2009-01-06  81.25  88.50  48.56  6.10   59.88   1.4919
6    2009-01-07  78.65  86.25  42.75  5.89   58.03   1.5111
7    2009-01-08  79.15  84.00  41.68  5.96   55.74   1.5202
...         ...    ...    ...    ...   ...     ...      ...
4254 2020-08-25  49.75  48.90  43.17  2.54   24.61   1.3148
4255 2020-08-26  49.70  48.95  43.21  2.52   25.10   1.3193
4256 2020-08-27  49.80  48.90  42.88  2.52   26.68   1.3206
4257 2020-08-28  50.60  52.20  42.96  2.46   28.95   1.3344
4260 2020-08-31  51.95  52.90  42.61  2.30   28.95   1.3379

[2890 rows x 7 columns]


In [8]:
## convert NBP pricing to $/MMBtu ##

df["NBP"] = (df["NBP_UK"]*0.1)*df["GBP-USD"]
df = df.dropna()
print(df["NBP"])

4       8.333553
5       8.933497
6       8.768913
7       8.473595
8       8.525626
          ...   
4254    3.235723
4255    3.311443
4256    3.523361
4257    3.863088
4260    3.873221
Name: NBP, Length: 2889, dtype: float64


In [9]:
## calculating price ratios ##

df['NBP_HH'] = df['NBP']/df['HH']
df['NBP_HH_r'] = df['NBP_HH'] - df['NBP_HH'].shift(1)
df = df.dropna()
print(df)
print(df["NBP_HH"])

           DATE    NCF    RCF    WTI    HH  NBP_UK  GBP-USD       NBP  \
5    2009-01-06  81.25  88.50  48.56  6.10   59.88   1.4919  8.933497   
6    2009-01-07  78.65  86.25  42.75  5.89   58.03   1.5111  8.768913   
7    2009-01-08  79.15  84.00  41.68  5.96   55.74   1.5202  8.473595   
8    2009-01-09  76.90  83.50  40.69  5.60   56.42   1.5111  8.525626   
11   2009-01-12  78.80  81.50  37.65  5.59   54.71   1.4809  8.102004   
...         ...    ...    ...    ...   ...     ...      ...       ...   
4254 2020-08-25  49.75  48.90  43.17  2.54   24.61   1.3148  3.235723   
4255 2020-08-26  49.70  48.95  43.21  2.52   25.10   1.3193  3.311443   
4256 2020-08-27  49.80  48.90  42.88  2.52   26.68   1.3206  3.523361   
4257 2020-08-28  50.60  52.20  42.96  2.46   28.95   1.3344  3.863088   
4260 2020-08-31  51.95  52.90  42.61  2.30   28.95   1.3379  3.873221   

        NBP_HH  NBP_HH_r  
5     1.464508  0.035082  
6     1.488780  0.024272  
7     1.421744 -0.067036  
8     1.522433 

In [ ]:
plt.plot(df["DATE"],df['HH'])
plt.plot(df["DATE"],df['NBP'])

In [7]:
## markov regime switching for Henry Hub and Newcastle Gas Futures##

mod = sm.tsa.MarkovRegression(df['NBP_HH'], k_regimes=2, order=1)
res_mod = mod.fit()
res_mod.summary()

/Users/joe/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/joe/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/regime_switching/markov_regression.py:307: RuntimeWarning: invalid value encountered in subtract
  variance = np.var(self.endog - np.dot(self.exog, beta))
/Users/joe/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/regime_switching/markov_regression.py:312: RuntimeWarning: invalid value encountered in multiply
  beta * (i / self.k_regimes))
/Users/joe/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/regime_switching/markov_regression.py:172: RuntimeWarning: invalid value encountered in subtract
  return self.endog - predict
/Users/joe/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization 

<class 'statsmodels.iolib.summary.Summary'>
"""
                        Markov Switching Model Results                        
==============================================================================
Dep. Variable:                 NBP_HH   No. Observations:                 2889
Model:               MarkovRegression   Log Likelihood                     nan
Date:                Tue, 23 Mar 2021   AIC                                nan
Time:                        11:36:43   BIC                                nan
Sample:                             0   HQIC                               nan
                               - 2889                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const             nan        nan        nan        nan         nan         nan
                             Regime 1 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const             nan        nan        nan        nan         nan         nan
                           Non-switching parameters                           
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2            nan        nan        nan        nan         nan         nan
                         Regime transition parameters                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p[0->0]           nan        nan        nan        nan         nan         nan
p[1->0]           nan        nan        nan        nan         nan         nan
==============================================================================

Warnings:
[1] Covariance matrix is singular or near-singular, with condition number    nan. Standard errors may be unstable.
"""

In [ ]:
res.smoothed_marginal_probabilities[3].plot(
    title='Probability of being in the high regime', figsize=(12,2))

In [ ]:
## OLS on volatility output ##

X = df['NBP']
y = df['WTI']
X = sm.add_constant(X)
mod1 = sm.OLS(y,X)
results = mod1.fit()
print(results.summary())

In [ ]:
sm.qqplot(results.resid,line="45",fit="TRUE")